In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
gdf = gpd.read_file(Path('data').joinpath('National_Forest_Inventory_Woodland_GB_2021.geojson'))

In [ ]:
gdf.columns

In [ ]:
gdf['CATEGORY'].unique()

In [ ]:
gdf['IFT_IOA'].unique()

In [ ]:
gdf.plot()

In [ ]:
woodlands = gdf[gdf['CATEGORY'] == 'Woodland']

In [ ]:
woodlands['IFT_IOA'].unique()

In [ ]:
fig = px.histogram(woodlands, x=woodlands['IFT_IOA'], text_auto=True)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Category", yaxis_title="Occurrences")

In [ ]:
area_sum_km2 = woodlands.groupby('IFT_IOA')['Area_ha'].sum()/100
fig = px.histogram(x=area_sum_km2.index, y=area_sum_km2.values.round(), text_auto=True)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Category", yaxis_title=f"Area in km\N{SUPERSCRIPT TWO}")

In [ ]:
area_sum_60by60m = woodlands.groupby('IFT_IOA')['Area_ha'].sum()/100/(0.06*0.06)
fig = px.histogram(x=area_sum_60by60m.index, y=area_sum_60by60m.values.round(), text_auto=True)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Category", yaxis_title='Area in arbitrary units', title=f"Area in units of 60 m by 60 m")

In [ ]:
binary_woodlands = woodlands[woodlands['IFT_IOA'].isin(['Broadleaved', 'Conifer'])]

# fig, ax = plt.subplots()
# binary_woodlands.plot(column='IFT_IOA', legend=True, cmap='jet', ax=ax)

In [ ]:
import datashader as ds
from matplotlib.cm import hot
import datashader.transfer_functions as tf
import seaborn

def plot_large_gpd(df, target='IFT_IOA'):
    canvas = ds.Canvas(plot_width=1000, plot_height=1000)
    reduced_data = canvas.polygons(df, geometry='geometry', agg=ds.count_cat(target))
    
    coords_lat, coords_lon = reduced_data.coords['y'].values, reduced_data.coords['x'].values
    coordinates = [
        [coords_lon[0], coords_lat[0]],
        [coords_lon[-1], coords_lat[0]],
        [coords_lon[-1], coords_lat[-1]],
        [coords_lon[0], coords_lat[-1]]]
    
    n_colors = seaborn.color_palette("dark:#5A9_r", df[target].nunique()).as_hex()
    color_key = dict(zip(df[target].unique(), n_colors))
    
    img = tf.shade(reduced_data, color_key=color_key)[::-1].to_pil()

    
    fig = px.scatter_mapbox(
                            lat=reduced_data.coords['y'].values, 
                            lon=reduced_data.coords['x'].values, 
                            zoom=4,width=1000, height=1000)
    
    fig.update_layout(
        mapbox_style="carto-darkmatter",
        mapbox_layers = [
            {
            "sourcetype": "image",
            "source": img,
            "coordinates": coordinates
            }]
    )
    return fig

df = binary_woodlands.copy()#.to_crs(crs='EPSG:3857')
df['IFT_IOA'] = df['IFT_IOA'].astype('category')
# fig = plot_large_gpd(df)

In [ ]:
fig

In [ ]:
canvas = ds.Canvas(plot_width=1000, plot_height=1000)
reduced_data = canvas.polygons(df, geometry='geometry', agg=ds.count_cat('IFT_IOA'))
color_key = dict(zip(df['IFT_IOA'].unique(), seaborn.color_palette("dark:#5A9_r", df['IFT_IOA'].nunique()).as_hex()))
img = tf.shade(reduced_data, color_key=color_key)[::-1].to_pil()
px.imshow(img, origin='lower', width=1000, height=1000)

In [ ]:
import os, colorcet, param as pm, holoviews as hv, panel as pn, datashader as ds
import intake
from holoviews.element import tiles as hvts
from holoviews.operation.datashader import rasterize, shade, spread

import holoviews.operation.datashader as hd

from collections import OrderedDict as odict

hv.extension('bokeh', logo=False)

In [ ]:
df.hvplot.scatter(x='longitude', y='latitude', rasterize=True, cnorm='eq_hist', dynspread=True)